<a href="https://colab.research.google.com/github/SkyW-PaulTischleder/dbt_salesforce/blob/master/Dayforce_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install dayforce-client

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 39.8 MB/s eta 0:00:00
  Created wheel for pysftp: filename=pysftp-0.2.9-py3-none-any.whl size=15496 sha256=aca1f5a96d22d09f8c2af31219357d3e6fead73a6ec99b78bad6e81ff54b11d3
  Stored in directory: /root/.cache/pip/wheels/64/52/87/2683826a06d053045a9bd11f05850e26bbdbfbf821ee6735f0
Successfully built pysftp


In [ ]:
pip install pandas

Get the list of reports with getReportMetadata
https://github.com/goodeggs/dayforce-client/blob/master/dayforce_client/client.py#L153


https://pypi.org/project/dayforce-client/


In [ ]:
import os

from dayforce_client.client import Dayforce


#date = datetime.now().strftime("%Y_%m_%d")
#snapshotdate = datetime.now().strftime("%Y-%m-%d")


DAYFORCE_USERNAME = 'datawarehouse.api'
DAYFORCE_PASSWORD = 'Winter123!'
DAYFORCE_CLIENT_NAMESPACE = 'skywater'


df = Dayforce(username=DAYFORCE_USERNAME,
              password=DAYFORCE_PASSWORD,
              client_namespace=DAYFORCE_CLIENT_NAMESPACE,
              dayforce_release="232")
#Dayforce(username='your-username', client_namespace='your-client-namespace', dayforce_release=57, api_version='V1', url='https://usr57-services.dayforcehcm.com/Api/your-client-namespace/V1')
#resp = df.get_employee_details(xrefcode='12458')
resp = df.get_reports()
for rawdata in resp.get("Data"):
  print(rawdata)
#print(resp.get("Data"))

{'Name': 'API_FinalPT', 'XRefCode': 'API_FinalPT', 'MaxRows': 20000}
{'Name': 'API_Employee_Census', 'XRefCode': 'API_Employee_Census', 'MaxRows': 20000}


Get Report Meta Data

get_report_metadata


In [ ]:
import os

from dayforce_client.client import Dayforce


#date = datetime.now().strftime("%Y_%m_%d")
#snapshotdate = datetime.now().strftime("%Y-%m-%d")

DAYFORCE_USERNAME = 'datawarehouse.api'
DAYFORCE_PASSWORD = 'Winter123!'
DAYFORCE_CLIENT_NAMESPACE = 'skywater'

df = Dayforce(username=DAYFORCE_USERNAME,
              password=DAYFORCE_PASSWORD,
              client_namespace=DAYFORCE_CLIENT_NAMESPACE,
              dayforce_release="231")

#Dayforce(username='your-username', client_namespace='your-client-namespace', dayforce_release=57, api_version='V1', url='https://usr57-services.dayforcehcm.com/Api/your-client-namespace/V1')
#resp = df.get_employee_details(xrefcode='12458')
resp = df.get_report_metadata(xrefcode='API_FinalPT')
for rawdata in resp.get("Data"):
  print(rawdata)

{'Name': 'API_FinalPT', 'XRefCode': 'API_FinalPT', 'MaxRows': 20000, 'ColumnMetadata': [{'CodeName': 'OrgUnit.ShortName', 'DisplayName': 'Location', 'DataType': 'String'}, {'CodeName': 'Department.DepartmentId', 'DisplayName': 'Department Id', 'DataType': 'String'}, {'CodeName': 'Department.ShortName', 'DisplayName': 'Department Name', 'DataType': 'String'}, {'CodeName': 'Employee.EmployeeId', 'DisplayName': 'Employee Number', 'DataType': 'String'}, {'CodeName': 'Employee.DisplayName', 'DisplayName': 'Employee Name', 'DataType': 'String'}, {'CodeName': 'Employee.Gender', 'DisplayName': 'Employee Gender', 'DataType': 'String'}, {'CodeName': 'DeptJob.JobId', 'DisplayName': 'Job Id', 'DataType': 'String'}, {'CodeName': 'DeptJob.ShortName', 'DisplayName': 'Job Assignment', 'DataType': 'String'}, {'CodeName': 'EmployeeManager.EmployeeManagerId', 'DisplayName': 'Employee Manager ID #', 'DataType': 'String'}, {'CodeName': 'EmployeeManager.ManagerDisplayName', 'DisplayName': 'Manager Name', 'D

Get the report Data


In [ ]:
import os
import pandas_gbq
import pandas as pd
from dayforce_client.client import Dayforce
from google.colab import auth
from google.cloud import bigquery

project = 'skywater-datalake-prod'
location = 'US'
client = bigquery.Client(project=project, location=location)

auth.authenticate_user()

#date = datetime.now().strftime("%Y_%m_%d")
#snapshotdate = datetime.now().strftime("%Y-%m-%d")
import datetime
from datetime import date
today = str(date.today())
from datetime import datetime
current_datetime = datetime.now()


#need to put these in Google seceret manager
DAYFORCE_USERNAME = 'datawarehouse.api'
DAYFORCE_PASSWORD = 'Winter123!'
DAYFORCE_CLIENT_NAMESPACE = 'skywater'

df = Dayforce(username=DAYFORCE_USERNAME,
              password=DAYFORCE_PASSWORD,
              client_namespace=DAYFORCE_CLIENT_NAMESPACE,
              dayforce_release="232")

#Call the API to get the report
resp = df.get_report(xrefcode='API_Employee_Census')

#load the JSON results into a pandas df
pdf = pd.DataFrame.from_dict(resp.get("Data"),orient='columns')

pdf2=pd.concat([pdf.XRefCode, pdf.Rows.apply(lambda x:pd.Series(x))],axis=1)

pdf2['SnapShotDate'] = today

pdf2.to_csv('/tmp/employee_census_' + today +'.csv', index=False)
#print(pdf2)
pandas_gbq.to_gbq(pdf2,'Dayforce.Employee_Census_Staging',project_id='skywater-datalake-prod',if_exists='replace')

# Copy the file to our new bucket.
!gsutil -m mv /tmp/employee_census_* gs://datalake-prod-uploads/dayforce/api-file-backup/

#print(resp.client)
#print(type(resp.client))
#print(resp.params)
#print(type(resp.resp))
#print(resp.resp.url)
#print(resp.resp.status_code)
#print(resp.resp.ok)
#print(resp.resp.elapsed)
#df.get_report(xrefcode='API_FinalPT', pageSize= '1')
#print(page)


100%|██████████| 1/1 [00:00<00:00, 1766.02it/s]


Copying file:///tmp/employee_census_2023-11-03.csv [Content-Type=text/csv]...
/ [1/1 files][747.0 KiB/747.0 KiB] 100% Done                                    
Operation completed over 1 objects/747.0 KiB.                                    
Removing file:///tmp/employee_census_2023-11-03.csv...


In [ ]:
### TESTING BOX
import os
import json
import pandas as pd


from dayforce_client.client import Dayforce

DAYFORCE_USERNAME = 'datawarehouse.api'
DAYFORCE_PASSWORD = 'Winter123!'
DAYFORCE_CLIENT_NAMESPACE = 'skywater'


df = Dayforce(username=DAYFORCE_USERNAME,
              password=DAYFORCE_PASSWORD,
              client_namespace=DAYFORCE_CLIENT_NAMESPACE,
              dayforce_release="231")
#for page, flip in df.get_employees().yield_records():
 #   print(flip)
#    print(page)

#Dayforce(username='your-username', client_namespace='your-client-namespace', dayforce_release=57, api_version='V1', url='https://usr57-services.dayforcehcm.com/Api/your-client-namespace/V1')
#resp = df.get_report(xrefcode='API_FinalPT',pageSize=10)
#resp = df.get_report(xrefcode='API_FinalPT')
json_object = df.get_report(xrefcode='API_FinalPT')
with open("/tmp/sample.json", "w") as outfile:
    outfile.write(json_object)

    import uuid

# Make a unique bucket to which we'll upload the file.
# (GCS buckets are part of a single global namespace.)
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/mb
!gsutil mb gs://datalake-prod-uploads/user_files

# Copy the file to our new bucket.
# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/cp
!gsutil cp /tmp/sample.json gs://datalake-prod-uploads/user_files

# Finally, dump the contents of our newly copied file to make sure everything worked.
!gsutil cat gs://datalake-prod-uploads/user_files/sample.json


#for raw_data in (resp.get("Data")):
 #   print(raw_data)
  #  print(resp.get("Data"))
#print(resp.get("Data"))



TypeError: ignored

https://pypi.org/project/dayforce-client/

https://github.com/goodeggs/dayforce-client/tree/master

